In [1]:
import numpy as np
import pandas as pd
from mat4py import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.utils import resample
from scipy import signal
import os
from scipy.linalg import circulant
import matplotlib.pyplot as plt
import random
import scipy
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn
from random import shuffle

In [2]:
file = open('D:\PhD topics\Datasets\Afib_datasets\store_no_noise_one_mean_final.pkl', 'rb')
#file= open('store_no_noise.pkl','rb')
obj = pickle.load(file)
file.close()

In [3]:
obj

{'ecg_train': array([[ 0.08608023,  0.08581871,  0.08469403, ...,  0.05728371,
          0.05974232,  0.06223159],
        [ 0.06668394,  0.06740808,  0.06773192, ...,  0.07126522,
          0.07099152,  0.07027662],
        [ 0.07094166,  0.07212875,  0.07197245, ...,  0.06147816,
          0.06396077,  0.06554425],
        ...,
        [-0.15816909, -0.17019585, -0.10206373, ..., -0.03658568,
         -0.04920873, -0.10357655],
        [-0.03617069, -0.02199404,  0.00623482, ..., -0.04964932,
         -0.03607034, -0.03592465],
        [-0.1316477 , -0.10271109, -0.11659159, ...,  0.01697524,
          0.07166445,  0.15376595]]),
 'ppg_train': array([[ 0.06185318,  0.06131563,  0.06131563, ...,  0.06239073,
          0.06239073,  0.06239073],
        [ 0.05970299,  0.05970299,  0.05970299, ...,  0.06024053,
          0.06024053,  0.05970299],
        [ 0.05378996,  0.05378996,  0.05325241, ...,  0.06077808,
          0.06131563,  0.06131563],
        ...,
        [ 0.79320376,  0.736

In [4]:
ecg_train=obj['ecg_train']
ecg_valid=obj['ecg_valid']
ecg_test=obj['ecg_test']
y_train=obj['y_train']
y_valid=obj['y_valid']
y_test=obj['y_test']

In [5]:
ecg_sqi_train=obj['ecg_sqi_train']
ecg_sqi_valid=obj['ecg_sqi_valid']
ecg_sqi_test=obj['ecg_sqi_test']

In [6]:
del obj

In [7]:
ecg_sqi_train_mean=np.mean(ecg_sqi_train, axis=0)
ecg_sqi_train_std=np.std(ecg_sqi_train, axis=0)

for i in range(ecg_sqi_train.shape[0]):
    ecg_sqi_train[i,:]=(ecg_sqi_train[i,:]-ecg_sqi_train_mean)/ecg_sqi_train_std
    
for i in range(ecg_sqi_valid.shape[0]):
    ecg_sqi_valid[i,:]=(ecg_sqi_valid[i,:]-ecg_sqi_train_mean)/ecg_sqi_train_std
    
for i in range(ecg_sqi_test.shape[0]):
    ecg_sqi_test[i,:]=(ecg_sqi_test[i,:]-ecg_sqi_train_mean)/ecg_sqi_train_std

In [8]:
ecg_train=np.expand_dims(ecg_train, axis=1)
ecg_valid=np.expand_dims(ecg_valid, axis=1)

ecg_sqi_train=np.expand_dims(ecg_sqi_train, axis=1)
ecg_sqi_valid=np.expand_dims(ecg_sqi_valid, axis=1)


In [9]:
ecg_test=np.expand_dims(ecg_test, axis=1)

ecg_sqi_test=np.expand_dims(ecg_sqi_test, axis=1)

In [10]:
num_classes=2
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [11]:
y_train_encoder = sklearn.preprocessing.LabelEncoder()
y_train_num = y_train_encoder.fit_transform(y_train)
y_train_wide = to_categorical(y_train_num, num_classes)

y_valid_num = y_train_encoder.fit_transform(y_valid)
y_valid_wide = to_categorical(y_valid_num, num_classes)

y_test_num = y_train_encoder.fit_transform(y_test)
y_test_wide = to_categorical(y_test_num, num_classes)

D:\conda_envs\torch_19\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [12]:
ecg_train = torch.from_numpy(ecg_train)
ecg_valid=torch.from_numpy(ecg_valid)
ecg_test=torch.from_numpy(ecg_test)


ecg_sqi_train = torch.from_numpy(ecg_sqi_train)
ecg_sqi_valid=torch.from_numpy(ecg_sqi_valid)
ecg_sqi_test = torch.from_numpy(ecg_sqi_test)

y_train_wide = torch.from_numpy(y_train_wide)
y_valid_wide = torch.from_numpy(y_valid_wide)
y_test_wide = torch.from_numpy(y_test_wide)
y_train_num = torch.from_numpy(y_train_num)
y_valid_num = torch.from_numpy(y_valid_num)
y_test_num = torch.from_numpy(y_test_num)

## ECG model

In [13]:
class ecg_model(nn.Module):
    def __init__(self):
        super().__init__()
        
        #ecg alone
        self.bn1_ecg=nn.BatchNorm1d(1)
        self.conv1_ecg = nn.Conv1d(in_channels=1, out_channels=5, kernel_size=30, stride=2)
        self.relu1_ecg = nn.LeakyReLU()
        self.max1_ecg = nn.MaxPool1d(2)
        self.drop1_ecg=nn.Dropout(0.25)
        self.conv2_ecg = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=25, stride=1)
        self.max2_ecg = nn.MaxPool1d(2)
        self.relu2_ecg = nn.LeakyReLU()
        self.drop2_ecg=nn.Dropout(0.25)
        self.conv3_ecg = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=20, stride=1)
        self.drop3_ecg=nn.Dropout(0.25)
        self.conv4_ecg = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=15, stride=1)
        self.max4_ecg = nn.MaxPool1d(2)
        self.relu4_ecg = nn.LeakyReLU()
        self.bn2_ecg=nn.BatchNorm1d(5)
        self.flat_ecg=nn.Flatten()
        self.drop4_ecg=nn.Dropout(0.25)
        self.linear1_ecg=nn.Linear(660,2)
        self.softm_ecg=nn.Softmax()
        
        
        
        
        
        
    
        
    def forward(self, ecg):
        
        ### ecg
        out_ecg_bn1=self.bn1_ecg(ecg)
        out_ecg_conv1= self.conv1_ecg(out_ecg_bn1)
        out_ecg_relu1 = self.relu1_ecg(out_ecg_conv1)
        out_ecg_max1=self.max1_ecg(out_ecg_relu1)
        out_ecg_drop1=self.drop1_ecg(out_ecg_max1)
        out_ecg_conv2=self.conv2_ecg(out_ecg_drop1)
        out_ecg_max2=self.max2_ecg(out_ecg_conv2)
        out_ecg_relu2=self.relu2_ecg(out_ecg_max2)
        out_ecg_drop2=self.drop2_ecg(out_ecg_relu2)
        out_ecg_conv3=self.conv3_ecg(out_ecg_drop2)
        out_ecg_drop3=self.drop3_ecg(out_ecg_conv3)
        out_ecg_conv4=self.conv4_ecg(out_ecg_drop3)
        out_ecg_max4=self.max4_ecg(out_ecg_conv4)
        out_ecg_relu4=self.relu4_ecg(out_ecg_max4)
        out_ecg_bn2=self.bn2_ecg(out_ecg_relu4)
        out_ecg_flat=self.flat_ecg(out_ecg_bn2)
        out_ecg_drop4=self.drop4_ecg(out_ecg_flat)
        out_ecg_linear=self.linear1_ecg(out_ecg_drop4)
        out_ecg_softm=self.softm_ecg(out_ecg_linear)
        
        
        return out_ecg_softm

In [14]:
model_ecg = ecg_model()

In [15]:
if torch.cuda.is_available():
    model_ecg = model_ecg.cuda()

In [16]:
# Define the loss function and optimizer
#loss_function = nn.functional.binary_cross_entropy
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_ecg.parameters(), lr=2e-5)
batch_size=32
n_batches=np.ceil(len(ecg_train)/batch_size).astype(np.int64)
valid_batch_size=100
valid_batches=np.ceil(len(ecg_valid)/valid_batch_size).astype(np.int64) 

min_valid_loss = np.inf

# Run the training loop
for epoch in range(0, 25): # 5 epochs at maximum
    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0
    current_tot_loss=0.0
    correct=0
    i=0
    # Iterate over the DataLoader for training data
    ind_list = list(range(len(ecg_train)))
    shuffle(ind_list)
    #train_new  = train[ind_list, :,:,:]
    #target_new = target[ind_list,]
    while i<n_batches:

          # Get inputs
        #local_X, local_y = X_train[i*batch_size:(i+1)*batch_size,], y_train_wide[i*batch_size:(i+1)*batch_size,]
        #local_X, local_y = X_train[i*batch_size:(i+1)*batch_size,], y_train_num[i*batch_size:(i+1)*batch_size,]
        local_ecg, local_y,  = ecg_train[ind_list[i*batch_size:(i+1)*batch_size],].cuda(),y_train_num[ind_list[i*batch_size:(i+1)*batch_size],].cuda()
        #if torch.cuda.is_available():
        #    local_X, local_y = local_X.cuda(), local_y.cuda()

          # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs_ecg =model_ecg(local_ecg.float())
        
        

        # Compute loss
       
        loss = loss_function(outputs_ecg, local_y)
        
        #loss=loss1
        #print(loss)
        
        #loss = loss_function(outputs, local_y.float())
        l2_lambda = 0.01
        l2_reg = torch.tensor(0.).cuda()
        for param in model_ecg.parameters():
            l2_reg += torch.norm(param)
        loss += l2_lambda * l2_reg
        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()
        
        output = outputs_ecg.clone()
        output= torch.argmax(output, axis=1)
        
        correct += (output == local_y).float().sum()

        # Print statistics
        
        #current_loss += loss.item()
        current_tot_loss+=loss.item()
       # if i % 20 == 19:
       #     print('Loss after mini-batch %5d: %.3f' %
       #         (i + 1, current_loss / 20))
       #     current_loss = 0.0
        i=i+1
        
    
    
    correct_valid=0
    current_valid_loss = 0.0
    current_valid_tot_loss=0.0
    j=0
    while j<valid_batches:
        model_ecg.eval()     # Optional when not using Model Specific layer
        local_ecg_valid, local_y_valid = ecg_valid[j*valid_batch_size:(j+1)*valid_batch_size,].cuda(),y_valid_num[j*valid_batch_size:(j+1)*valid_batch_size,].cuda()
        target = model_ecg(local_ecg_valid.float())
        loss1 = loss_function(target,local_y_valid)
        
        loss=loss1
        
        current_valid_loss+=loss.item()
        #valid_loss = loss.item() 
        target= torch.argmax(target, axis=1)
        correct_valid += (target == local_y_valid).float().sum()
        j=j+1
    
    #print(correct)
    #print(current_tot_loss)
    accuracy = 100 * correct / len(ecg_train)
    print(f'Epoch {epoch+1} \t\t Training Loss: {current_tot_loss / n_batches} \t \t Accuracy = {accuracy}, \t Validation Loss: {current_valid_loss/ valid_batches}, \t Validation accuracy: {100*correct_valid/len(ecg_valid)}')
    
    if min_valid_loss > (current_valid_loss/valid_batches):
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{current_valid_loss/valid_batches:.6f}) \t Saving The Model')
        min_valid_loss = (current_valid_loss/valid_batches)
        # Saving State Dict
        torch.save(model_ecg.state_dict(), 'saved_model_ecg_no_noise_0.01.pth')
        
    
# Process is complete.
print('Training process has finished.')

Starting epoch 1


D:\conda_envs\torch_19\lib\site-packages\torch\nn\functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)
D:\conda_envs\torch_19\lib\site-packages\ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1 		 Training Loss: 0.6347301948913618 	 	 Accuracy = 77.17759704589844, 	 Validation Loss: 0.7854875727006566, 	 Validation accuracy: 50.50516891479492
Validation Loss Decreased(inf--->0.785488) 	 Saving The Model
Starting epoch 2
Epoch 2 		 Training Loss: 0.5448765151370737 	 	 Accuracy = 86.65608978271484, 	 Validation Loss: 0.4369614350865459, 	 Validation accuracy: 87.7525863647461
Validation Loss Decreased(0.785488--->0.436961) 	 Saving The Model
Starting epoch 3
Epoch 3 		 Training Loss: 0.5259714604362398 	 	 Accuracy = 87.83245849609375, 	 Validation Loss: 0.4340469042460124, 	 Validation accuracy: 87.92293548583984
Validation Loss Decreased(0.436961--->0.434047) 	 Saving The Model
Starting epoch 4
Epoch 4 		 Training Loss: 0.5188017133693386 	 	 Accuracy = 87.98152160644531, 	 Validation Loss: 0.4325793458704363, 	 Validation accuracy: 88.11677551269531
Validation Loss Decreased(0.434047--->0.432579) 	 Saving The Model
Starting epoch 5
Epoch 5 		 Training Loss: 0.498184

In [17]:
model_saved = ecg_model()
model_saved.load_state_dict(torch.load('saved_model_ecg_no_noise_0.01.pth'))
model_saved.eval()
model_saved=model_saved.cpu()

In [18]:
outputs_test = model_saved(ecg_test.float())
outputs_test = outputs_test.clone()
outputs_test= torch.argmax(outputs_test, axis=1)

print("ECG output")
        
correct = (outputs_test == y_test_num).float().sum()
print(correct/len(ecg_test))

cm=confusion_matrix(y_test_num.cpu(), outputs_test.cpu())
print(cm)



D:\conda_envs\torch_19\lib\site-packages\ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


ECG output
tensor(0.9596)
[[7996  515]
 [  49 5399]]
